In [1]:
import pandas as pd
from datetime import datetime
from datetime import timedelta
import numpy as np
from scipy.stats.stats import pearsonr
import matplotlib.pyplot as plt
import json
from sklearn import preprocessing

In [2]:
playtest5_data = pd.read_csv("2018-11-14_Playtest/anonymized_playtest5_data.csv", sep=";")
playtest6_data = pd.read_csv("2019-01-07_Playtest/anonymized_playtest6_data.csv", sep=";")
playtest7_data = pd.read_csv("2019-01-31_Playtest/anonymized_playtest7_data.csv", sep=";")

In [3]:
import collections

puzzles = collections.OrderedDict()
with open("2019-01-07_Playtest/StreamingAssets/config.json") as f:
    asset_config = json.load(f)

for puzzle_sets in asset_config.get("puzzleSets"):
    for puzzle_file in puzzle_sets.get("puzzles"):
        with open(f"2019-01-07_Playtest/StreamingAssets/{puzzle_file}.json") as f:
            puzzle_details = json.load(f)
            print(puzzle_details.get("puzzleName"))
            puzzles[puzzle_details.get("puzzleName")] = puzzle_details 
            
sequence_puzzles = dict()
i = 1
for puzzle_id in puzzles.keys():
    sequence_puzzles[puzzle_id] = i
    i += 1

One Cube
Separated Cubes
Rotate a Pyramid
Match Silhouettes
Removing Objects
Stretch a Ramp
Max 2 Cubes
Stack 2 Ramps
Scaling Round Objects
Square Cross-Sections
Bird Fez
Pi Henge
45-Degree Rotations
Pyramids are Strange
Cubes Obscure Spheres
Shape Limits
Not Bird
Angled Silhouette
Design a Puzzle


In [4]:
playtest5_data['time'] = pd.to_datetime(playtest5_data['time'])
playtest6_data['time'] = pd.to_datetime(playtest6_data['time'])
playtest7_data['time'] = pd.to_datetime(playtest7_data['time'])

In [5]:
playtest5_data = playtest5_data.sort_values('time')
playtest6_data = playtest6_data.sort_values('time')
playtest7_data = playtest7_data.sort_values('time')

In [6]:
all_playtest_data = pd.concat([playtest5_data, playtest6_data, playtest7_data])

In [7]:
all_playtest_data = all_playtest_data.sort_values('time')

# Difficulty measure

In [8]:
listMovementEvents = ['ws-move_shape', 'ws-rotate_shape', 'ws-scale_shape', 
                      'ws-check_solution', 'ws-undo_action', 'ws-redo_action',
                      'ws-rotate_view', 'ws-snapshot']

In [9]:
# puzzleDict[user~puzzle_id] = {}
puzzleDict = {}
theresHoldActivity = 15

for user in playtest6_data['session_id'].unique():

    user_events = playtest6_data[playtest6_data['session_id'] == user]
    # Analyze when a puzzle has been started
    activePuzzle = None
    previousEvent = None
    numberActions = 0
    numberAttempts = 0
    activeTime = []

    for enum, event in user_events.iterrows():
        print(('{} - {}').format(event['time'], event['type']))
        if(event['type'] == 'ws-start_level'):
            print('\\start level\\')
            print(json.loads(event['data']))
            activePuzzle = json.loads(event['data'])['task_id']

        # If event is puzzle complete we always add it
        if(event['type'] == 'ws-puzzle_complete'):
            key = event['session_id'] + '~' + json.loads(event['data'])['task_id']
            if(key in puzzleDict.keys()):
                puzzleDict[key]['n_complete'] += 1

        # If they are not playing a puzzle we do not do anything and continue
        if(activePuzzle is None):
            continue

        key = event['session_id'] + '~' + activePuzzle
        if(key not in puzzleDict.keys()):
            puzzleDict[key] = dict()
            puzzleDict[key]['active_time'] = 0
            puzzleDict[key]['n_actions'] = 0
            puzzleDict[key]['n_attempts'] = 0
            puzzleDict[key]['n_complete'] = 0

        # If it is the first event we store the current event and continue
        if(previousEvent is None):
            previousEvent = event
            continue

        delta_seconds = (event['time'] - previousEvent['time']).total_seconds()
        if(~(delta_seconds > theresHoldActivity)):
            activeTime.append(delta_seconds)

        if(event['type'] in listMovementEvents):
            numberActions += 1

        if(event['type'] == 'ws-check_solution'):
            numberAttempts += 1

    # Analyze when puzzle is finished or user left
        # Measure time, attempts, completion and actions
        if(event['type'] in ['ws-puzzle_complete', 'ws-exit_to_menu', 'ws-disconnect']):
            print('\\finish\\')
            # time spent
            print('{} minutes, {} actions, {} attempts'.format(round(np.sum(activeTime)/60,2), numberActions, numberAttempts))
            # adding counters
            puzzleDict[key]['active_time'] += round(np.sum(activeTime)/60,2)
            puzzleDict[key]['n_actions'] += numberActions
            puzzleDict[key]['n_attempts'] += numberAttempts

            # reset counters
            previousEvent = None
            activeTime = []
            activePuzzle = None
            numberActions = 0
            numberAttempts = 0

        previousEvent = event
        

2019-01-07 09:21:56.348838 - ws-start_game
2019-01-07 09:21:56.690505 - ws-login_user
2019-01-07 09:22:34.038821 - ws-start_game
2019-01-07 09:22:34.308816 - ws-login_user
2019-01-07 09:22:40.118394 - ws-start_game
2019-01-07 09:22:40.421342 - ws-login_user
2019-01-07 09:23:28.347133 - ws-start_game
2019-01-07 09:23:28.942914 - ws-login_user
2019-01-07 09:25:52.420854 - ws-start_game
2019-01-07 09:25:52.865741 - ws-login_user
2019-01-07 09:29:19.056075 - ws-start_game
2019-01-07 09:29:19.590240 - ws-login_user
2019-01-07 09:29:25.459990 - ws-start_level
\start level\
{'user': 'guest', 'group': '', 'task_id': 'Design a Puzzle', 'set_id': 'Design a Puzzle', 'fullscreen': False, 'resolution': {'x': 960, 'y': 540}, 'conditions': '{"shapeLimits":[3,1,1,1,1,1,1],"allowScale":true,"allowRotate":true,"gridDim":5}'}
2019-01-07 09:29:26.062998 - ws-puzzle_started
2019-01-07 09:29:26.441748 - ws-click_nothing
2019-01-07 09:29:38.924302 - ws-mode_change
2019-01-07 09:29:41.548499 - ws-mode_change


2019-01-07 14:20:24.186709 - ws-rotate_shape
2019-01-07 14:20:41.943588 - ws-check_solution
2019-01-07 14:20:42.278904 - ws-puzzle_complete
\finish\
6.08 minutes, 78 actions, 1 attempts
2019-01-07 14:20:52.705618 - ws-rotate_view
2019-01-07 14:20:54.231800 - ws-rotate_view
2019-01-07 14:20:55.715167 - ws-deselect_shape
2019-01-07 14:21:00.152563 - ws-rotate_view
2019-01-07 14:21:03.125062 - ws-rotate_view
2019-01-07 14:21:04.907911 - ws-rotate_view
2019-01-07 14:21:08.248506 - ws-snapshot
2019-01-07 14:21:14.367768 - ws-check_solution
2019-01-07 14:21:14.538316 - ws-puzzle_complete
2019-01-07 14:21:15.647737 - ws-exit_to_menu
2019-01-07 14:21:19.489743 - ws-start_level
\start level\
{'user': 'guest', 'group': '', 'task_id': 'Square Cross-Sections', 'set_id': 'Puzzles', 'fullscreen': True, 'resolution': {'x': 1366, 'y': 768}, 'conditions': '{"shapeLimits":[3,1,1,1,1,1,1],"allowScale":true,"allowRotate":true,"gridDim":5}'}
2019-01-07 14:21:19.912321 - ws-puzzle_started
2019-01-07 14:21:2

2019-01-07 14:09:30.523514 - ws-create_shape
2019-01-07 14:09:33.601518 - ws-mode_change
2019-01-07 14:09:35.420593 - ws-rotate_shape
2019-01-07 14:09:37.923325 - ws-rotate_shape
2019-01-07 14:09:41.001332 - ws-rotate_shape
2019-01-07 14:09:44.193212 - ws-deselect_shape
2019-01-07 14:09:45.765758 - ws-mode_change
2019-01-07 14:09:47.285258 - ws-select_shape
2019-01-07 14:09:48.662693 - ws-scale_shape
2019-01-07 14:09:51.282047 - ws-deselect_shape
2019-01-07 14:09:53.031976 - ws-mode_change
2019-01-07 14:09:54.239180 - ws-select_shape
2019-01-07 14:09:55.272109 - ws-move_shape
2019-01-07 14:10:31.702456 - ws-deselect_shape
2019-01-07 14:10:34.228893 - ws-rotate_view
2019-01-07 14:10:37.770035 - ws-rotate_view
2019-01-07 14:10:42.608906 - ws-mode_change
2019-01-07 14:10:43.416923 - ws-create_shape
2019-01-07 14:10:50.760108 - ws-mode_change
2019-01-07 14:10:51.733812 - ws-create_shape
2019-01-07 14:10:51.919287 - ws-deselect_shape
2019-01-07 14:10:52.334028 - ws-select_shape
2019-01-07 1

2019-01-07 14:00:54.087975 - ws-mode_change
2019-01-07 14:00:55.702394 - ws-create_shape
2019-01-07 14:00:56.046248 - ws-deselect_shape
2019-01-07 14:00:56.800341 - ws-select_shape
2019-01-07 14:00:58.067115 - ws-rotate_shape
2019-01-07 14:00:58.236014 - ws-rotate_shape
2019-01-07 14:01:01.304542 - ws-rotate_shape
2019-01-07 14:01:09.821973 - ws-deselect_shape
2019-01-07 14:01:12.820644 - ws-rotate_view
2019-01-07 14:01:17.005060 - ws-rotate_view
2019-01-07 14:01:20.825968 - ws-rotate_view
2019-01-07 14:01:22.052552 - ws-rotate_view
2019-01-07 14:01:23.833417 - ws-mode_change
2019-01-07 14:01:25.002356 - ws-create_shape
2019-01-07 14:01:26.206086 - ws-deselect_shape
2019-01-07 14:01:42.071885 - ws-check_solution
2019-01-07 14:01:42.584911 - ws-puzzle_complete
\finish\
1.77 minutes, 19 actions, 1 attempts
2019-01-07 14:01:45.642946 - ws-exit_to_menu
2019-01-07 14:01:48.584148 - ws-start_level
\start level\
{'user': 'guest', 'group': '', 'task_id': 'Cubes Obscure Spheres', 'set_id': 'Puz

2019-01-07 14:02:14.630250 - ws-move_shape
2019-01-07 14:02:15.458051 - ws-move_shape
2019-01-07 14:02:16.129507 - ws-move_shape
2019-01-07 14:02:18.248961 - ws-mode_change
2019-01-07 14:02:20.181213 - ws-mode_change
2019-01-07 14:02:25.672876 - ws-rotate_shape
2019-01-07 14:02:27.372629 - ws-rotate_shape
2019-01-07 14:02:28.047963 - ws-rotate_shape
2019-01-07 14:02:28.225617 - ws-rotate_shape
2019-01-07 14:02:29.619237 - ws-rotate_shape
2019-01-07 14:02:29.976396 - ws-rotate_shape
2019-01-07 14:02:36.889952 - ws-deselect_shape
2019-01-07 14:02:38.441897 - ws-rotate_view
2019-01-07 14:02:40.560130 - ws-rotate_view
2019-01-07 14:02:42.875541 - ws-rotate_view
2019-01-07 14:02:44.043202 - ws-rotate_view
2019-01-07 14:02:46.751584 - ws-rotate_view
2019-01-07 14:02:47.989886 - ws-rotate_view
2019-01-07 14:02:49.517087 - ws-rotate_view
2019-01-07 14:02:55.089065 - ws-rotate_view
2019-01-07 14:02:57.559750 - ws-rotate_view
2019-01-07 14:02:59.948997 - ws-rotate_view
2019-01-07 14:03:05.103344

...

2019-01-07 13:51:28.883887 - ws-check_solution
2019-01-07 13:51:29.052979 - ws-puzzle_complete
\finish\
1.66 minutes, 21 actions, 1 attempts
2019-01-07 13:51:31.134116 - ws-exit_to_menu
2019-01-07 13:51:38.024512 - ws-start_level
\start level\
{'user': 'guest', 'group': '', 'task_id': 'Square Cross-Sections', 'set_id': 'Puzzles', 'fullscreen': True, 'resolution': {'x': 1366, 'y': 768}, 'conditions': '{"shapeLimits":[3,1,1,1,1,1,1],"allowScale":true,"allowRotate":true,"gridDim":5}'}
2019-01-07 13:51:38.481196 - ws-puzzle_started
2019-01-07 13:51:42.248709 - ws-mode_change
2019-01-07 13:51:59.188789 - ws-mode_change
2019-01-07 13:52:09.047314 - ws-create_shape
2019-01-07 13:52:10.283936 - ws-mode_change
2019-01-07 13:52:11.240024 - ws-create_shape
2019-01-07 13:52:14.835826 - ws-mode_change
2019-01-07 13:52:17.777891 - ws-create_shape
2019-01-07 13:52:39.688679 - ws-rotate_view
2019-01-07 13:52:44.315404 - ws-snapshot
2019-01-07 13:52:47.370327 - ws-rotate_view
2019-01-07 13:52:51.179624

2019-01-07 14:16:05.123933 - ws-deselect_shape
2019-01-07 14:16:11.989936 - ws-mode_change
2019-01-07 14:16:13.476871 - ws-select_shape
2019-01-07 14:16:14.960052 - ws-scale_shape
2019-01-07 14:16:15.671847 - ws-select_shape
2019-01-07 14:16:18.774723 - ws-scale_shape
2019-01-07 14:16:21.372669 - ws-select_shape
2019-01-07 14:16:22.055643 - ws-scale_shape
2019-01-07 14:16:22.376465 - ws-scale_shape
2019-01-07 14:16:22.727642 - ws-scale_shape
2019-01-07 14:16:23.060257 - ws-scale_shape
2019-01-07 14:16:23.754635 - ws-scale_shape
2019-01-07 14:16:24.427886 - ws-scale_shape
2019-01-07 14:16:24.597100 - ws-scale_shape
2019-01-07 14:16:25.261414 - ws-scale_shape
2019-01-07 14:16:25.621403 - ws-select_shape
2019-01-07 14:16:26.184809 - ws-scale_shape
2019-01-07 14:16:26.565102 - ws-scale_shape
2019-01-07 14:16:26.727390 - ws-scale_shape
2019-01-07 14:16:27.063350 - ws-scale_shape
2019-01-07 14:16:27.408145 - ws-scale_shape
2019-01-07 14:16:27.607993 - ws-scale_shape
2019-01-07 14:16:27.97139

2019-01-07 14:12:57.743683 - ws-puzzle_started
2019-01-07 14:13:00.411291 - ws-mode_change
2019-01-07 14:13:02.475983 - ws-create_shape
2019-01-07 14:13:05.146530 - ws-deselect_shape
2019-01-07 14:13:07.966639 - ws-undo_action
2019-01-07 14:13:09.461638 - ws-redo_action
2019-01-07 14:13:11.535716 - ws-rotate_view
2019-01-07 14:13:13.106453 - ws-mode_change
2019-01-07 14:13:16.163391 - ws-create_shape
2019-01-07 14:13:17.639787 - ws-mode_change
2019-01-07 14:13:18.486875 - ws-rotate_shape
2019-01-07 14:13:19.064580 - ws-rotate_shape
2019-01-07 14:13:20.244716 - ws-rotate_shape
2019-01-07 14:13:20.496325 - ws-rotate_shape
2019-01-07 14:13:21.269904 - ws-deselect_shape
2019-01-07 14:13:22.172146 - ws-click_nothing
2019-01-07 14:13:22.712758 - ws-mode_change
2019-01-07 14:13:23.315626 - ws-select_shape
2019-01-07 14:13:24.567222 - ws-deselect_shape
2019-01-07 14:13:25.754199 - ws-rotate_view
2019-01-07 14:13:26.505810 - ws-select_shape
2019-01-07 14:13:29.069643 - ws-move_shape
2019-01-07 

2019-01-07 13:45:19.250828 - ws-deselect_shape
2019-01-07 13:45:20.683040 - ws-click_nothing
2019-01-07 13:45:25.220617 - ws-undo_action
2019-01-07 13:45:25.602609 - ws-undo_action
2019-01-07 13:45:26.600674 - ws-undo_action
2019-01-07 13:45:27.537619 - ws-undo_action
2019-01-07 13:45:28.367978 - ws-deselect_shape
2019-01-07 13:45:30.261033 - ws-rotate_view
2019-01-07 13:45:31.868385 - ws-rotate_view
2019-01-07 13:45:33.370904 - ws-rotate_view
2019-01-07 13:45:45.344499 - ws-check_solution
2019-01-07 13:45:49.060479 - ws-select_shape
2019-01-07 13:45:50.447489 - ws-deselect_shape
2019-01-07 13:45:51.601390 - ws-rotate_view
2019-01-07 13:45:56.915298 - ws-click_nothing
2019-01-07 13:46:00.564243 - ws-click_nothing
2019-01-07 13:46:05.726516 - ws-rotate_view
2019-01-07 13:46:07.022583 - ws-rotate_view
2019-01-07 13:46:09.353354 - ws-select_shape
2019-01-07 13:46:11.581871 - ws-rotate_view
2019-01-07 13:46:21.590977 - ws-deselect_shape
2019-01-07 13:46:22.550939 - ws-select_shape
2019-01-

2019-01-07 14:10:25.127652 - ws-select_shape
2019-01-07 14:10:27.208469 - ws-move_shape
2019-01-07 14:10:42.099584 - ws-check_solution
2019-01-07 14:10:48.462410 - ws-select_shape
2019-01-07 14:10:50.030686 - ws-select_shape
2019-01-07 14:10:53.087776 - ws-mode_change
2019-01-07 14:10:54.204703 - ws-mode_change
2019-01-07 14:10:54.847898 - ws-delete_shape
2019-01-07 14:10:55.195144 - ws-select_shape
2019-01-07 14:10:56.144249 - ws-mode_change
2019-01-07 14:10:57.310011 - ws-delete_shape
2019-01-07 14:10:59.369855 - ws-mode_change
2019-01-07 14:11:00.606155 - ws-create_shape
2019-01-07 14:11:05.216968 - ws-mode_change
2019-01-07 14:11:06.186537 - ws-create_shape
2019-01-07 14:11:07.535998 - ws-move_shape
2019-01-07 14:11:08.424311 - ws-move_shape
2019-01-07 14:11:09.742343 - ws-mode_change
2019-01-07 14:11:12.372040 - ws-rotate_shape
2019-01-07 14:11:12.963331 - ws-rotate_shape
2019-01-07 14:11:16.362263 - ws-mode_change
2019-01-07 14:11:17.669501 - ws-create_shape
2019-01-07 14:11:18.8

In [10]:
puzzleDf = pd.DataFrame.from_dict(puzzleDict, orient='index')

In [11]:
puzzleDf['session_id'] = [item[0] for item in puzzleDf.index.str.split('~')]
puzzleDf['puzzle_id'] = [item[1] for item in puzzleDf.index.str.split('~')]

In [12]:
puzzleDf

,active_time,n_actions,n_attempts,n_complete,session_id,puzzle_id
3xasusf0vzfwny3xivt3gg1xwxx4bcd3-Playtest6~Angled Silhouette,10.35,157,4,1,3xasusf0vzfwny3xivt3gg1xwxx4bcd3-Playtest6,Angled Silhouette
3xasusf0vzfwny3xivt3gg1xwxx4bcd3-Playtest6~Cubes Obscure Spheres,26.85,55,1,0,3xasusf0vzfwny3xivt3gg1xwxx4bcd3-Playtest6,Cubes Obscure Spheres
3xasusf0vzfwny3xivt3gg1xwxx4bcd3-Playtest6~Design a Puzzle,0.24,0,0,0,3xasusf0vzfwny3xivt3gg1xwxx4bcd3-Playtest6,Design a Puzzle
3xasusf0vzfwny3xivt3gg1xwxx4bcd3-Playtest6~Match Silhouettes,2.60,59,0,0,3xasusf0vzfwny3xivt3gg1xwxx4bcd3-Playtest6,Match Silhouettes
3xasusf0vzfwny3xivt3gg1xwxx4bcd3-Playtest6~Not Bird,5.20,46,1,0,3xasusf0vzfwny3xivt3gg1xwxx4bcd3-Playtest6,Not Bird
3xasusf0vzfwny3xivt3gg1xwxx4bcd3-Playtest6~One Cube,0.63,1,1,1,3xasusf0vzfwny3xivt3gg1xwxx4bcd3-Playtest6,One Cube
3xasusf0vzfwny3xivt3gg1xwxx4bcd3-Playtest6~Rotate a Pyramid,2.42,57,3,1,3xasusf0vzfwny3xivt3gg1xwxx4bcd3-Playtest6,Rotate a Pyramid
3xasusf0vzfwny3xivt3gg1xwxx4bcd3-Playtest6~Sandbox,0.20,0,0,0,3xasusf0vzfwny3xivt3gg1xwxx4bcd3-Playtest6,Sandbox
3xasusf0vzfwny3xivt3gg1xwxx4bcd3-Playtest6~Separated Cubes,0.82,2,1,1,3xasusf0vzfwny3xivt3gg1xwxx4bcd3-Playtest6,Separated Cubes
3xasusf0vzfwny3xivt3gg1xwxx4bcd3-Playtest6~Shape Limits,3.31,32,3,1,3xasusf0vzfwny3xivt3gg1xwxx4bcd3-Playtest6,Shape Limits


In [13]:
puzzleDf['p_incorrect'] = 100-100*(puzzleDf['n_complete']/puzzleDf['n_attempts'])

In [14]:
data_by_puzzle = puzzleDf.groupby('puzzle_id').agg({'active_time': 'mean',
                                   'n_attempts': 'mean',
                                   'n_actions': 'mean',
                                   'p_incorrect': 'mean',
                                   'n_complete': 'sum'}).reset_index()

In [15]:
data_by_puzzle['order'] = data_by_puzzle['puzzle_id'].map(sequence_puzzles) 

In [16]:
data_by_puzzle.sort_values('order', inplace=True)

In [17]:
data_by_puzzle['z_active_time'] = (data_by_puzzle['active_time'] - data_by_puzzle['active_time'].mean())/data_by_puzzle['active_time'].std()
data_by_puzzle['z_n_actions'] = (data_by_puzzle['n_actions'] - data_by_puzzle['n_actions'].mean())/data_by_puzzle['n_actions'].std()
data_by_puzzle['z_p_incorrect'] = (data_by_puzzle['p_incorrect'] - data_by_puzzle['p_incorrect'].mean())/data_by_puzzle['p_incorrect'].std()

In [18]:
data_by_puzzle['z_all_measures'] = data_by_puzzle[['z_active_time', 'z_n_actions', 'z_p_incorrect']].sum(axis = 1)

In [19]:
data_by_puzzle['norm_all_measures'] = (data_by_puzzle['z_all_measures']-data_by_puzzle['z_all_measures'].min())/(data_by_puzzle['z_all_measures'].max()-data_by_puzzle['z_all_measures'].min())

In [20]:
data_by_puzzle.to_csv('difficulty_gba.csv', index = False)

In [21]:
data_by_puzzle

,puzzle_id,active_time,n_attempts,n_actions,p_incorrect,n_complete,order,z_active_time,z_n_actions,z_p_incorrect,z_all_measures,norm_all_measures
8,One Cube,0.468182,0.909091,0.909091,0.000000,10,1.0,-0.941043,-1.301868,-1.159090,-3.402001,0.000000
15,Separated Cubes,0.788000,1.300000,4.500000,12.962963,10,2.0,-0.859636,-1.127466,-0.635465,-2.622567,0.078606
12,Rotate a Pyramid,9.471000,2.700000,43.100000,37.424242,10,3.0,1.350561,0.747245,0.352622,2.450428,0.590215
5,Match Silhouettes,15.850833,1.166667,11.250000,16.666667,10,4.0,2.974503,-0.799634,-0.485858,1.689011,0.513426
11,Removing Objects,1.203333,1.222222,4.222222,16.666667,9,5.0,-0.753916,-1.140957,-0.485858,-2.380731,0.102995
19,Stretch a Ramp,1.453333,1.777778,11.000000,30.555556,9,6.0,-0.690280,-0.811776,0.075169,-1.426888,0.199189
6,Max 2 Cubes,1.517778,1.555556,9.666667,27.777778,8,7.0,-0.673876,-0.876533,-0.037037,-1.587446,0.182997
18,Stack 2 Ramps,1.712222,0.888889,22.111111,0.000000,8,8.0,-0.624382,-0.272136,-1.159090,-2.055608,0.135783
14,Scaling Round Objects,4.753750,2.000000,51.875000,43.750000,7,9.0,0.149818,1.173426,0.608144,1.931388,0.537870
17,Square Cross-Sections,3.780000,1.333333,25.777778,15.625000,8,10.0,-0.098043,-0.094054,-0.527935,-0.720033,0.270475
